In [1]:
from tensorflow.keras.layers import (Conv2D, UpSampling2D, Conv2DTranspose, concatenate, MaxPooling2D, GlobalAveragePooling2D, 
                                     Activation, Dropout, Cropping2D, Flatten, Dense, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import vgg16
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
import efficientnet.tfkeras as efn

Using TensorFlow backend.


In [2]:
IMG_HEIGHT = 300
IMG_WIDTH = 300
TRAIN_IMAGE_PATH = "jpeg/train/"
TEST_IMAGE_PATH = "jpeg/test/"
BATCH_SIZE = 8
AUTO = tf.data.experimental.AUTOTUNE
REG = 0.0005
EPOCHS=100
METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.binary_crossentropy,
]

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [4]:
train_df["image_name"] = train_df["image_name"].apply(lambda x: TRAIN_IMAGE_PATH + x + ".jpg")
test_df["image_name"] = test_df["image_name"].apply(lambda x: TEST_IMAGE_PATH + x + ".jpg")

In [5]:
train_df_target_1 = train_df[train_df["target"] == 1]
train_df_target_0 = train_df[train_df["target"] == 0]

In [6]:
del train_df

In [7]:
random_target_0 = np.random.randint(low=1, high=train_df_target_0.shape[0], size=10 * train_df_target_1.shape[0])

In [8]:
train_df_d = pd.concat([train_df_target_0.iloc[random_target_0], train_df_target_1])

In [9]:
def decode_image(filename, label=None, image_size=(IMG_WIDTH, IMG_HEIGHT)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=image_size)
    
    if label is None:
        return image
    else:
        return image, label
#image = tf.cast(image, tf.float32)/255.0
    
def data_augment(image, label=None):
    r_crop = np.random.uniform(low = 0.4, high = 1.0)
    r_rsize = np.random.uniform(low = 0.8, high = 1.2)
    image = tf.image.random_crop(image, (int(r_crop*IMG_HEIGHT), int(r_crop*IMG_WIDTH), 3))
    image = tf.image.rot90(image)
    image = tf.keras.preprocessing.image.random_shear(image, 20)
    image = tf.image.resize(image, (int(r_rsize*IMG_HEIGHT), int(r_rsize*IMG_WIDTH), 3), preserve_aspect_ratio=True)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_saturation(image, lower=0.7, upper=1.3)
    image = tf.image.random_contrast(image, lower=0.7, upper=1.3)
    image = tf.image.random_brightness(image, lower=0.7, upper=1.3)
    image = tf.image.rgb_to_hsv(image)
    if label is None:
        return image
    else:
        return image, label

In [10]:
# x_train = train_df[["image_name"]]
# y_train = train_df["target"].astype(np.float32).values
# x_train.reset_index(drop=True, inplace=True)

In [11]:
x_train = train_df_d[["image_name"]]
y_train = train_df_d["target"].astype(np.float32).values
x_train.reset_index(drop=True, inplace=True)

In [12]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=45)

In [13]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((5139, 1), (1285, 1), (5139,), (1285,))

In [14]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

In [15]:
class_weights

array([0.55009634, 5.49038462])

In [16]:
train_dataset = (tf.data.Dataset
                 .from_tensor_slices((x_train["image_name"].values, y_train))
                 .map(decode_image, num_parallel_calls=AUTO)
                 .repeat()
                 .shuffle(512)
                 .batch(BATCH_SIZE)
                 .prefetch(AUTO)
                )

In [17]:
val_dataset = (tf.data.Dataset
               .from_tensor_slices((x_val["image_name"].values, y_val))
               .map(decode_image, num_parallel_calls=AUTO)
               .repeat()
               .shuffle(512)
               .batch(BATCH_SIZE)
               .prefetch(AUTO))

In [18]:
test_dataset = (tf.data.Dataset.from_tensor_slices((test_df.image_name))
                .map(decode_image, num_parallel_calls=AUTO)
                .cache()
                .batch(BATCH_SIZE))

In [19]:
lr = ReduceLROnPlateau(
    monitor="val_auc",
    patience=3,
    min_lr=0.000001,
    factor=0.5,
    verbose=1
)

In [20]:
es = EarlyStopping(monitor="val_loss", patience=10)

In [21]:
model_chkpt = ModelCheckpoint(filepath="best_model.h5")

In [22]:

def get_crop_shape(target, refer):
        # width, the 3rd dimension
        print("target: {} {}, refer: {} {}".format(target, target.get_shape(), refer, refer.get_shape()))
        cw = (target.get_shape()[2] - refer.get_shape()[2])
        assert (cw >= 0)
        if cw % 2 != 0:
            cw1, cw2 = int(cw/2), int(cw/2) + 1
        else:
            cw1, cw2 = int(cw/2), int(cw/2)
        # height, the 2nd dimension
        ch = (target.get_shape()[1] - refer.get_shape()[1])
        assert (ch >= 0)
        if ch % 2 != 0:
            ch1, ch2 = int(ch/2), int(ch/2) + 1
        else:
            ch1, ch2 = int(ch/2), int(ch/2)

        return (ch1, ch2), (cw1, cw2)

In [23]:
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
    denominator = tf.reduce_sum(y_true + y_pred, axis=-1)
    return 1 - (numerator + 1) / (denominator + 1)

In [24]:
def create_model(input_layer):
    
    pretrained_model = efn.EfficientNetB3(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                                              weights='imagenet',
                                              include_top=False
                                             )
    # False = transfer learning, True = fine-tuning
    pretrained_model.trainable = True#False 

    inp1 = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name='inp1')
        

    x=pretrained_model(inp1)
    x=tf.keras.layers.GlobalAveragePooling2D()(x)
    x=tf.keras.layers.Dense(2048, 
                                kernel_regularizer=tf.keras.regularizers.l2(l=REG),
                                activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
    x=tf.keras.layers.Dense(1024, 
                                kernel_regularizer=tf.keras.regularizers.l2(l=REG),
                                activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
    x=tf.keras.layers.Dense(512, 
                                kernel_regularizer=tf.keras.regularizers.l2(l=REG),
                                activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
    x=tf.keras.layers.Dense(256, 
                                kernel_regularizer=tf.keras.regularizers.l2(l=REG),
                                activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
    x=tf.keras.layers.Dense(128, 
                                kernel_regularizer=tf.keras.regularizers.l2(l=REG),
                                activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
    x=tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(l=REG),
                                activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
        
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)
        
    model = tf.keras.models.Model(inputs=[inp1], outputs=[output])
    
    model.compile(
        optimizer='adam',
        loss = dice_loss,
        metrics=[tf.keras.metrics.AUC()],
        )
        
    return model

In [25]:
input_layer = Input((IMG_HEIGHT, IMG_WIDTH, 3))
model = create_model(input_layer=input_layer)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp1 (InputLayer)            [(None, 300, 300, 3)]     0         
_________________________________________________________________
efficientnet-b3 (Model)      (None, 10, 10, 1536)      10783528  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              3147776   
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0     

In [26]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=dice_loss, metrics=METRICS)

In [27]:
history = model.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, es, model_chkpt],
                        steps_per_epoch=x_train.shape[0]//BATCH_SIZE, validation_data=val_dataset,
                        validation_steps=x_val.shape[0]//BATCH_SIZE,
                        class_weight = class_weights)

Train for 642 steps, validate for 160 steps
Epoch 1/100
642/642 [==============================] - 738s 1s/step - loss: 1.0809 - tp: 0.0000e+00 - fp: 2.0000 - tn: 4668.0000 - fn: 466.0000 - accuracy: 0.9089 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5153 - binary_crossentropy: 0.9735 - val_loss: 0.3249 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 1169.0000 - val_fn: 111.0000 - val_accuracy: 0.9133 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5468 - val_binary_crossentropy: 0.7003
Epoch 2/100
642/642 [==============================] - 674s 1s/step - loss: 0.1687 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 4666.0000 - fn: 470.0000 - accuracy: 0.9085 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5054 - binary_crossentropy: 1.0264 - val_loss: 0.0937 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 1165.0000 - val_fn: 115.0000 - val_accuracy: 0.9102 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5115 - val_binary_crossentropy:

641/642 [============================>.] - ETA: 0s - loss: 0.0479 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 4652.0000 - fn: 476.0000 - accuracy: 0.9072 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.6041 - binary_crossentropy: 0.7382
Epoch 00016: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.
642/642 [==============================] - 654s 1s/step - loss: 0.0479 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 4659.0000 - fn: 477.0000 - accuracy: 0.9071 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.6049 - binary_crossentropy: 0.7376 - val_loss: 0.0490 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 1159.0000 - val_fn: 121.0000 - val_accuracy: 0.9055 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6178 - val_binary_crossentropy: 0.7244
Epoch 17/100
642/642 [==============================] - 644s 1s/step - loss: 0.0468 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 4670.0000 - fn: 466.0000 - accuracy: 0.9093 - precision: 0.0000e+00 - recall: 0.0000e

In [28]:
history

In [29]:
pred = model.predict(test_dataset)

In [30]:
pred

array([[6.3005409e-06],
       [3.5176547e-06],
       [2.7030246e-06],
       ...,
       [4.9238235e-01],
       [6.8428158e-06],
       [8.6698653e-03]], dtype=float32)

In [31]:
sam = pd.read_csv("sample_submission.csv")

In [32]:
sam.head()


,image_name,target
0,ISIC_0052060,0
1,ISIC_0052349,0
2,ISIC_0058510,0
3,ISIC_0073313,0
4,ISIC_0073502,0


In [33]:
sam["target"] = pred

In [34]:
sam

,image_name,target
0,ISIC_0052060,0.000006
1,ISIC_0052349,0.000004
2,ISIC_0058510,0.000003
3,ISIC_0073313,0.000007
4,ISIC_0073502,0.000019
...,...,...
10977,ISIC_9992485,0.000013
10978,ISIC_9996992,0.000034
10979,ISIC_9997917,0.492382
10980,ISIC_9998234,0.000007


In [35]:
sam.to_csv("efnb3_barzil_researcher_image_augmentation_exp_4x_imbalance_data_2048_cells_100_epochs.csv", index=False)